## Mini Project III

In [67]:
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import pandas as pd

first 8 of acct_nbr are uniform.

acct_nbr contains no duplicates :)

one years worth of transactions

33 states, 69 cities

Investigate clustering by 1st postal code

In [68]:
customer = pd.read_csv('data/twm_customer.csv', delimiter=';')

account = pd.read_csv('data/twm_accounts.csv', delimiter=';')
transactions = pd.read_csv('data/twm_transactions.csv', delimiter=';')

account_checking = pd.read_csv('data/twm_checking_acct.csv', delimiter=';')
account_credit = pd.read_csv('data/twm_credit_acct.csv', delimiter=';')
account_saving = pd.read_csv('data/twm_savings_acct.csv', delimiter=';')

transaction_checking = pd.read_csv('data/twm_checking_tran.csv', delimiter=';')
transaction_credit = pd.read_csv('data/twm_credit_tran.csv', delimiter=';')
transaction_saving = pd.read_csv('data/twm_savings_tran.csv', delimiter=';')

# random push test hello ////////////////////////

# Cleaning
### First
* strip white spaces from str columns
* convert dates to computable date types, and times to just the hour.
* make account numbers uniform
* drop columns



In [69]:
all_dfs = [customer,account,transactions,account_checking,account_credit,account_saving,transaction_checking,transaction_credit,transaction_saving]

for i in range(len(all_dfs)):
    for col in all_dfs[i].select_dtypes('object'):
        all_dfs[i][col] = all_dfs[i][col].str.strip()

customer,account,transactions,account_checking,account_credit,account_saving,transaction_checking,transaction_credit,transaction_saving = all_dfs

tran_dfs = [transactions,transaction_checking,transaction_credit,transaction_saving]
for i in range(len(tran_dfs)):
    tran_dfs[i]['tran_time'] = tran_dfs[i]['tran_time'].astype(str)
    tran_dfs[i] = tran_dfs[i].replace('',np.nan)
    tran_dfs[i][(tran_dfs[i]['tran_time'].str.len() > 6) | (tran_dfs[i]['tran_time'].str.len() <=4)] = np.nan
    tran_dfs[i]['tran_time'] = pd.to_datetime(tran_dfs[i]['tran_time'], format='%H%M%S').dt.hour
    tran_dfs[i]['tran_date'] = pd.to_datetime(tran_dfs[i]['tran_date'], format='%d.%M.%Y').dt.normalize()

transactions,transaction_checking,transaction_credit,transaction_saving = tran_dfs

account_number_dfs =[account,transactions,account_credit]

for df in account_number_dfs:
    df['acct_nbr'] = df['acct_nbr'].astype(str)

    df['acct_nbr'] = np.where(df['acct_nbr'].str.len() == 16, df['acct_nbr'].str[8:],df['acct_nbr'])

account,transactions,account_credit = account_number_dfs

In [70]:
customer = customer.drop(['first_name','last_name','street_nbr','street_name'],axis=1)

In [71]:
account

,acct_nbr,cust_id,acct_type,account_active,acct_start_date,acct_end_date,starting_balance,ending_balance
0,13628063,1362806,SV,Y,10.12.1995,NaN,1430.22,284.58
1,13627090,1362709,CC,Y,15.3.1993,NaN,266.34,496.15
2,13628360,1362836,CC,Y,18.3.1992,NaN,55.90,1000.00
3,13633112,1363311,CK,Y,6.7.1995,NaN,11017.13,968.46
4,13633610,1363361,CC,Y,17.6.1994,NaN,849.37,462.28
...,...,...,...,...,...,...,...,...
1404,13625560,1362556,CC,N,20.1.1992,19.8.1995,1433.94,20.00
1405,13626240,1362624,CC,Y,29.11.1990,NaN,632.75,1400.00
1406,13628750,1362875,CC,Y,24.2.1989,NaN,2810.91,3000.00
1407,13626543,1362654,SV,Y,17.10.1992,NaN,784.53,622.46


In [72]:
customer.city_name.value_counts().count()

69

In [73]:
customer.state_code.value_counts().count()

33

In [74]:
for col in transactions.select_dtypes('object'):
    transactions[col] = transactions[col].str.strip()

In [75]:
account.head()

,acct_nbr,cust_id,acct_type,account_active,acct_start_date,acct_end_date,starting_balance,ending_balance
0,13628063,1362806,SV,Y,10.12.1995,NaN,1430.22,284.58
1,13627090,1362709,CC,Y,15.3.1993,NaN,266.34,496.15
2,13628360,1362836,CC,Y,18.3.1992,NaN,55.90,1000.00
3,13633112,1363311,CK,Y,6.7.1995,NaN,11017.13,968.46
4,13633610,1363361,CC,Y,17.6.1994,NaN,849.37,462.28


In [76]:
account_checking.head()

,cust_id,acct_nbr,minimum_balance,per_check_fee,account_active,acct_start_date,acct_end_date,starting_balance,ending_balance
0,1362548,13625482,3000,0.00,Y,11.11.1986,NaN,6004.34,569.65
1,1362487,13624872,3000,0.00,Y,12.1.1995,NaN,2781.07,1401.21
2,1363017,13630172,3000,0.00,Y,19.3.1994,NaN,2694.91,147.15
3,1362752,13627522,100,0.15,Y,8.1.1994,NaN,487.69,3.76
4,1363282,13632822,100,0.15,Y,5.3.1988,NaN,133.90,84.18


In [77]:
transactions[transactions['tran_id'] == 26]

,tran_id,acct_nbr,tran_amt,principal_amt,interest_amt,new_balance,tran_date,tran_time,channel,tran_code
193,26.0,13632842.0,182.12,182.12,0.0,202.77,1995-01-11,23.0,A,DP
300,26.0,4561143213624840.0,-158.95,-158.95,0.0,-368.11,1995-01-04,13.0,E,CG
384,26.0,13626312.0,-225.16,-225.16,0.0,1867.99,1995-01-29,17.0,E,WD
613,26.0,4561143213629390.0,-438.64,-438.64,0.0,-1243.37,1995-01-03,16.0,E,CG
767,26.0,4561143213631870.0,-76.12,-76.12,0.0,-2172.27,1995-01-05,15.0,E,CG
...,...,...,...,...,...,...,...,...,...,...
77265,26.0,13634212.0,0.00,0.00,0.0,2435.78,1995-01-02,11.0,B,IQ
77302,26.0,13632422.0,148.19,148.19,0.0,158.99,1995-01-18,18.0,B,DP
77315,26.0,13629912.0,268.71,268.71,0.0,323.29,1995-01-13,23.0,A,DP
77320,26.0,13626072.0,0.00,0.00,0.0,63.95,1995-01-08,8.0,A,IQ


In [78]:
account[account['cust_id'] == 1363017]

,acct_nbr,cust_id,acct_type,account_active,acct_start_date,acct_end_date,starting_balance,ending_balance
766,13630172,1363017,CK,Y,19.3.1994,NaN,2694.91,147.15
949,13630170,1363017,CC,Y,8.2.1995,NaN,0.00,1000.00
1191,13630173,1363017,SV,N,27.4.1994,31.10.1995,2011.02,2901.35


In [ ]:
for i in range(100):
    print('Test')

In [ ]:
# Lasagna

Spaghetti